In [225]:
# coding:utf-8
import numpy as np
import pandas as pd
from pandas import Series, DataFrame

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split


%matplotlib inline


In [226]:
train_user = pd.read_csv('data/table1_user',encoding='utf-8',index_col = False, delimiter='\t',
                                 )
train_user['cur_jd_type'] = train_user['cur_jd_type'].astype(str)
train_user.loc[train_user['cur_jd_type']=='nan',['cur_jd_type']] = '无业'
train_job = pd.read_csv("data/table2_jd",delimiter="\t",error_bad_lines=False)
train_action = pd.read_csv("data/table3_action",delimiter="\t")
test_action = pd.read_csv("data/zhaopin_round1_user_exposure_B_20190819",delim_whitespace=True)
test_user = pd.read_csv("data/user_ToBePredicted",delimiter="\t")
test_user['cur_jd_type'] = test_user['cur_jd_type'].astype(str)
test_user.loc[test_user['cur_jd_type']=='nan',['cur_jd_type']] = '无业'
train_action['mix_sat']  = train_action['satisfied']*10+train_action['delivered']*3
train_action.loc[train_action.mix_sat==13,'mix_sat'] =10

b'Skipping line 173315: expected 18 fields, saw 20\n'
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [227]:
train_big_table = pd.merge(train_action,train_user,how="inner",on="user_id")
train_big_table = pd.merge(train_big_table,train_job,how="inner",on="jd_no")
test_big_table = pd.merge(test_action,test_user,how="inner",on="user_id")
test_big_table = pd.merge(test_big_table,train_job,how="inner",on="jd_no")
train_big_table.drop_duplicates(inplace=True)
test_big_table.drop_duplicates(inplace=True)

In [228]:
train_big_table['statis_del'] = train_big_table['satisfied']
train_big_table.loc[(train_big_table['statis_del']==0)&(train_big_table['delivered']==1),['statis_del']]=0.3

In [229]:
jd_sub_series = train_big_table['jd_sub_type'].value_counts()
train_big_table.loc[~train_big_table['jd_sub_type'].isin(jd_sub_series[jd_sub_series>30].index.tolist()),['jd_sub_type']]='小众'
test_big_table.loc[~test_big_table['jd_sub_type'].isin(jd_sub_series[jd_sub_series>30].index.tolist()),['jd_sub_type']]='小众'




In [230]:

jd_title_series = train_big_table['jd_title'].value_counts()
train_big_table.loc[~train_big_table['jd_title'].isin(jd_title_series[jd_title_series>30].index.tolist()),['jd_title']]='小众'
test_big_table.loc[~test_big_table['jd_title'].isin(jd_title_series[jd_title_series>30].index.tolist()),['jd_title']]='小众'



In [231]:

desire_jd_type_id_series = train_big_table['desire_jd_type_id'].value_counts()
train_big_table.loc[~train_big_table['desire_jd_type_id'].isin(desire_jd_type_id_series[desire_jd_type_id_series>30].index.tolist()),['desire_jd_type_id']]='小众'
test_big_table.loc[~test_big_table['desire_jd_type_id'].isin(desire_jd_type_id_series[desire_jd_type_id_series>30].index.tolist()),['desire_jd_type_id']]='小众'



In [232]:
desire_jd_industry_series = train_big_table['desire_jd_industry_id'].value_counts()
train_big_table.loc[~train_big_table['desire_jd_industry_id'].isin(desire_jd_industry_series[desire_jd_industry_series>30].index.tolist()),['desire_jd_industry_id']]='小众'
test_big_table.loc[~test_big_table['desire_jd_industry_id'].isin(desire_jd_industry_series[desire_jd_industry_series>30].index.tolist()),['desire_jd_industry_id']]='小众'


In [233]:
cur_jd_action_series = train_big_table.groupby(['cur_jd_type'])['browsed'].count()
cur_jd_action =pd.DataFrame({'cur_jd_type':cur_jd_action_series.index, 'cur_jd_pos_cont':cur_jd_action_series.values})
cur_jd_action['cur_jd_statis_del_rate'] = train_big_table.groupby(['cur_jd_type'])['statis_del'].mean().values
test_big_table = pd.merge(test_big_table,cur_jd_action,how="left",on="cur_jd_type")
test_big_table.loc[test_big_table['cur_jd_pos_cont'].isnull(),['cur_jd_statis_del_rate']]=cur_jd_action[cur_jd_action['cur_jd_type'].str.contains('其他')][['cur_jd_statis_del_rate']].values[0]

In [234]:
jd_sub_action_series = train_big_table.groupby(['jd_sub_type'])['browsed'].count()
jd_sub_action =pd.DataFrame({'jd_sub_type':jd_sub_action_series.index, 'jd_sub_pos_cont':jd_sub_action_series.values})
jd_sub_action['jd_sub_statis_del_rate'] = train_big_table.groupby(['jd_sub_type'])['statis_del'].mean().values
test_big_table = pd.merge(test_big_table,jd_sub_action,how="left",on="jd_sub_type")
test_big_table.loc[test_big_table['jd_sub_pos_cont'].isnull(),['jd_sub_statis_del_rate']] =jd_sub_action[jd_sub_action['jd_sub_type'].str.contains('小众')][['jd_sub_statis_del_rate']].values[0]




In [235]:

desire_jd_industry_action_series = train_big_table.groupby(['desire_jd_type_id'])['browsed'].count()
desire_jd_industry_action =pd.DataFrame({'desire_jd_type_id':desire_jd_industry_action_series.index, 'desire_jd_industry_pos_cont':desire_jd_industry_action_series.values})
desire_jd_industry_action['desire_jd_industry_statis_del_rate'] = train_big_table.groupby(['desire_jd_type_id'])['statis_del'].mean().values
test_big_table = pd.merge(test_big_table,desire_jd_industry_action,how="left",on="desire_jd_type_id")
test_big_table.loc[test_big_table['desire_jd_industry_pos_cont'].isnull(),['desire_jd_industry_statis_del_rate']]=desire_jd_industry_action[desire_jd_industry_action['desire_jd_type_id'].str.contains('小众')][['desire_jd_industry_statis_del_rate']].values[0]




In [236]:


jd_title_series = train_big_table.groupby(['jd_title'])['browsed'].count()
jd_title_action =pd.DataFrame({'jd_title':jd_title_series.index, 'cur_jd_pos_cont':jd_title_series.values})
jd_title_action['jd_title_statis_del_rate'] = train_big_table.groupby(['jd_title'])['statis_del'].mean().values
test_big_table = pd.merge(test_big_table,jd_title_action,how="left",on="jd_title")
test_big_table.loc[test_big_table['jd_title'].isnull(),['jd_title_statis_del_rate']]=jd_title_action[jd_title_action['jd_title'].str.contains('小众')][['jd_title_statis_del_rate']].values[0]




In [237]:
'''
desire_jd_type_action_series = train_big_table.groupby(['desire_jd_type_id'])['browsed'].count()
desire_jd_type_action =pd.DataFrame({'desire_jd_type_id':desire_jd_type_action_series.index, 'desire_jd_type_pos_cont':desire_jd_type_action_series.values})
desire_jd_type_action['desire_jd_type_statis_del_rate'] = train_big_table.groupby(['desire_jd_type_id'])['statis_del'].mean().values
test_big_table = pd.merge(test_big_table,desire_jd_type_action,how="left",on="desire_jd_type_id")
test_big_table.loc[test_big_table['desire_jd_type_pos_cont'].isnull(),['desire_jd_type_statis_del_rate']] =desire_jd_type_action[desire_jd_type_action['desire_jd_type_id'].str.contains('小众')][['desire_jd_type_statis_del_rate']].values[0]
'''




'\ndesire_jd_type_action_series = train_big_table.groupby([\'desire_jd_type_id\'])[\'browsed\'].count()\ndesire_jd_type_action =pd.DataFrame({\'desire_jd_type_id\':desire_jd_type_action_series.index, \'desire_jd_type_pos_cont\':desire_jd_type_action_series.values})\ndesire_jd_type_action[\'desire_jd_type_statis_del_rate\'] = train_big_table.groupby([\'desire_jd_type_id\'])[\'statis_del\'].mean().values\ntest_big_table = pd.merge(test_big_table,desire_jd_type_action,how="left",on="desire_jd_type_id")\ntest_big_table.loc[test_big_table[\'desire_jd_type_pos_cont\'].isnull(),[\'desire_jd_type_statis_del_rate\']] =desire_jd_type_action[desire_jd_type_action[\'desire_jd_type_id\'].str.contains(\'小众\')][[\'desire_jd_type_statis_del_rate\']].values[0]\n'

In [238]:
train_big_table['start_date'].fillna('na',inplace=True)
test_big_table['start_date'].fillna('na',inplace=True)
train_big_table['end_date'].fillna('na',inplace=True)
test_big_table['end_date'].fillna('na',inplace=True)
train_big_table['start_date'] = train_big_table['start_date'].astype(str)
train_big_table['end_date'] = train_big_table['end_date'].astype(str)
test_big_table['start_date'] = test_big_table['start_date'].astype(str)
test_big_table['end_date'] = test_big_table['end_date'].astype(str)

In [239]:
from datetime import datetime

def remain_date(x):
    if  'N' in x :
        return 99999;
    elif 'na' in x:
        return -1
    else:
        d1 = datetime.strptime('20190327', "%Y%m%d")
        d2 = datetime.strptime(x, "%Y%m%d")
        return abs((d2 - d1).days)
    
train_big_table['remain_date'] = train_big_table['end_date'].apply(remain_date)
test_big_table['remain_date'] = test_big_table['end_date'].apply(remain_date)

In [240]:
def month_convert(x):
    if  'N' in x:
        return -1;
    elif 'na' in x:
        return -1
    else:
        if int(x[0:6])< 201804:
            return '201804'
        else:
            return x[0:6]

train_big_table['start_month'] = train_big_table['start_date'].apply(month_convert)
test_big_table['start_month'] = test_big_table['start_date'].apply(month_convert)

In [241]:
def extract_city(citys,index):
    city_list = citys.split(",")
    if index < len(city_list):
        city = city_list[index]
        if city != "-":
            city = int(city)
        else:
            city = -1
    else:
        city = -1
    return city

def exp_in_desc(exp,desc):
    if str(exp) == "nan":
        exp = ""
    exps = exp.split("|")
    num = 0
    for item in exps:
        if item in desc:
            num+=1
    return num

In [242]:
min_year_dict = {
    305: 4,
    -1:-1,
    1:1,
    103:2,
    0:0,
    510:7,
    1099:10,
    399:4,
    599:7,
    199:1,
    299:2,
    110:1
}
degree_dict = {
    "初中":1,
    "中技":2,
    "高中":3,
    "中专":3,
    "大专":4,
    "本科":5,
    "硕士":6,
    "博士":7,
    "EMBA":7,
    "MBA":6,
    "其他":0,
    "请选择":0,
    "\\N":0,
    "na":0
}
min_salary_dict = {
    100002000:1000,
    400106000:4001,
    0:0,
    200104000:2001,
    600108000:6001,
    800110000:8001,
    1000115000:10001,
    2500199999:25001,
    1500125000:15001,
    3500150000:35001,
    70001100000:70001,
    1000:0,
    100001150000:100001,
    2500135000:25001,
    5000170000:50001
}
max_salary_dict = {
    100002000:2000,
    400106000:6000,
    0:0,
    200104000:4000,
    600108000:8000,
    800110000:10000,
    1000115000:15000,
    2500199999:99999,
    1500125000:25000,
    3500150000:50000,
    70001100000:100000,
    1000:1000,
    100001150000:150000,
    2500135000:35000,
    5000170000:70000
}

In [243]:
from functools import partial
def fe(df):
    df["desire_jd_city_1"] = df["desire_jd_city_id"].apply(partial(extract_city,index=0))
    df["desire_jd_city_2"] = df["desire_jd_city_id"].apply(partial(extract_city,index=1))
    df["desire_jd_city_3"] = df["desire_jd_city_id"].apply(partial(extract_city,index=2))
    df["desire_jd_city_num"] = df[["desire_jd_city_1","desire_jd_city_2","desire_jd_city_3"]].sum(axis=1)
    
    df["city_equal_desired_city_1"] = df["desire_jd_city_1"]==df["city"]
    df["city_equal_desired_city_2"] = df["desire_jd_city_2"]==df["city"]
    df["city_equal_desired_city_3"] = df["desire_jd_city_3"]==df["city"]
    
    df["work_years"] = 2019-df["start_work_date"].apply(lambda x : 2018 if x=="-" else int(x))
    
    
    df["desire_min_salary"] = df["desire_jd_salary_id"].apply(lambda x: min_salary_dict[x])
    df["desire_max_salary"] = df["desire_jd_salary_id"].apply(lambda x: max_salary_dict[x])
    df["desire_salary_diff"] = df["desire_max_salary"]-df["desire_min_salary"]
    
    df["min_years"] = df["min_years"].apply(lambda x: min_year_dict[x])
    
    df["work_years_statisfied"] = df["work_years"].astype(int) > df["min_years"]
    
    df["salary_large_than_desire"] = df["desire_min_salary"] > df["min_salary"]
    
    df["cur_salary_min"] = df["cur_salary_id"].apply(lambda x: min_salary_dict[int(x if str.isnumeric(x) else "0")])
    df["cur_salary_max"] = df["cur_salary_id"].apply(lambda x: max_salary_dict[int(x if str.isnumeric(x) else "0")])
    
    df["salary_large_than_cur"] = df["cur_salary_min"] > df["min_salary"]
    
    df["cur_degree_id"] = df["cur_degree_id"].fillna("na").apply(lambda x:degree_dict[x.strip()])
    
    df["job_description_len"] = df["job_description"].apply(len)
    
    df["experience_num"] = df["experience"].apply(lambda x: len(str(x).split("|")) if str(x) != "nan" else 0)
    
    df["min_edu_level"] = df["min_edu_level"].fillna("na").apply(lambda x:degree_dict[x.strip()])
    exp_in_desc_num = []
    for idx, data in df.iterrows():
        exp_in_desc_num.append(exp_in_desc(data["experience"],data["job_description"]))
    df["exp_in_desc_num"] = exp_in_desc_num
    
#     "live_city_id","desire_jd_salary_id","cur_industry_id","cur_jd_type","cur_salary_id",
#          "cur_degree_id","city","jd_sub_type",
#          "max_salary","min_salary","is_travel","min_years","min_edu_level",
#          "desire_jd_city_1","desire_jd_city_2","desire_jd_city_3","work_years_statisfied"
#  "jd_sub_browsed_dicret","jd_sub_delivered_dicret","jd_sub_satisfied_dicret"  
    cross_feature_tuple = [("live_city_id","city"),("live_city_id","desire_jd_city_1"),("cur_industry_id","jd_sub_type"),
                          ("cur_jd_type","jd_sub_type"),("cur_salary_id","cur_degree_id"),("city","jd_sub_type"),
                          ("jd_sub_type","min_salary"),("jd_sub_type","max_salary"),("jd_sub_type","is_travel"),
                          ("min_years","jd_sub_type"),("jd_sub_type","require_nums"),
                          ]
    cross_feature_names = list(feature[0]+"&"+feature[1] for feature in cross_feature_tuple)
    print("create cross features",cross_feature_names)
    for idx,(fa,fb) in enumerate(cross_feature_tuple):
        df[cross_feature_names[idx]] = df[fa].astype(str)+df[fb].astype(str)
    return cross_feature_names

In [244]:
cross_feature_names = fe(train_big_table)
fe(test_big_table)

create cross features ['live_city_id&city', 'live_city_id&desire_jd_city_1', 'cur_industry_id&jd_sub_type', 'cur_jd_type&jd_sub_type', 'cur_salary_id&cur_degree_id', 'city&jd_sub_type', 'jd_sub_type&min_salary', 'jd_sub_type&max_salary', 'jd_sub_type&is_travel', 'min_years&jd_sub_type', 'jd_sub_type&require_nums']
create cross features ['live_city_id&city', 'live_city_id&desire_jd_city_1', 'cur_industry_id&jd_sub_type', 'cur_jd_type&jd_sub_type', 'cur_salary_id&cur_degree_id', 'city&jd_sub_type', 'jd_sub_type&min_salary', 'jd_sub_type&max_salary', 'jd_sub_type&is_travel', 'min_years&jd_sub_type', 'jd_sub_type&require_nums']


['live_city_id&city',
 'live_city_id&desire_jd_city_1',
 'cur_industry_id&jd_sub_type',
 'cur_jd_type&jd_sub_type',
 'cur_salary_id&cur_degree_id',
 'city&jd_sub_type',
 'jd_sub_type&min_salary',
 'jd_sub_type&max_salary',
 'jd_sub_type&is_travel',
 'min_years&jd_sub_type',
 'jd_sub_type&require_nums']

In [245]:
all_big_table = pd.concat([train_big_table,test_big_table])

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  if __name__ == '__main__':


In [246]:
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm as tqdm
import lightgbm as lgb

In [252]:
def feature_select(target,*df_list):
    result = []
    cat_features = ["live_city_id","desire_jd_salary_id","cur_industry_id","cur_jd_type","cur_salary_id",
         "cur_degree_id","city","jd_sub_type","start_month",
         "min_salary","is_travel","min_years","min_edu_level",
         "desire_jd_city_1","desire_jd_city_2","desire_jd_city_3","work_years_statisfied"]+cross_feature_names
    lbl_dict = {}
    for f in cat_features:
        lbl = LabelEncoder()
        lbl.fit(all_big_table[f].astype(str))
        lbl_dict[f] = lbl
    for df in df_list:
        features = ["live_city_id","desire_jd_salary_id","cur_industry_id","cur_jd_type","cur_salary_id",
             "cur_degree_id","birthday","city","jd_sub_type","require_nums","jd_sub_statis_del_rate","cur_jd_statis_del_rate",
                    "start_month","jd_title_statis_del_rate",
             "min_salary","is_travel","min_years","min_edu_level",
             "desire_jd_city_1","desire_jd_city_2","desire_jd_city_3","exp_in_desc_num",
                   "desire_min_salary","desire_max_salary","salary_large_than_desire","cur_salary_min",
                   "cur_salary_max","salary_large_than_cur","job_description_len","experience_num","work_years_statisfied","work_years","desire_jd_city_num","desire_salary_diff"]+cross_feature_names

        x = df[features]
        if target in df.columns:
            y = df[target]
        else:
            y = None
        for f in cat_features:
            lbl = lbl_dict[f]
            x[f] = lbl.transform(x[f].astype(str))
        result.append((x,y))
    return result


In [253]:
from sklearn.model_selection import KFold

def cross_validate(param=dict(n_estimators=1000,metric="map",colsample_bytree=0.2,max_depth=7,importance_type="gain",device="gpu",gpu_platform_id=0,gpu_device_id=0)
                   ,n_folds=5,target="mix_sat"):
    train_users = train_big_table["user_id"].unique()
    folds = KFold(n_folds,shuffle=True,random_state=42)
    models = []
    test_pred = np.zeros(test_big_table.shape[0])
    scores = []
    for idx,(train_idx,valid_idx) in enumerate(folds.split(train_users)):
        t_user = train_users[train_idx]
        v_user = train_users[valid_idx]
        train_data = train_big_table[train_big_table["user_id"].isin(t_user)]
        valid_data = train_big_table[train_big_table["user_id"].isin(v_user)]
        train_group = train_data.groupby("user_id",as_index=False).count()["mix_sat"].values
        valid_group = valid_data.groupby("user_id",as_index=False).count()["mix_sat"].values
        test_group = test_big_table.groupby("user_id",as_index=False).count()["jd_no"].values
        
        
        
        jd_sub_action_series = train_data.groupby(['jd_sub_type'])['browsed'].count()
        jd_sub_action =pd.DataFrame({'jd_sub_type':jd_sub_action_series.index, 'jd_sub_pos_cont':jd_sub_action_series.values})
        jd_sub_action['jd_sub_statis_del_rate'] = train_data.groupby(['jd_sub_type'])['statis_del'].mean().values
        train_data = pd.merge(train_data,jd_sub_action,how="left",on="jd_sub_type")
        valid_data = pd.merge(valid_data,jd_sub_action,how="left",on="jd_sub_type")
        valid_data.loc[valid_data['jd_sub_pos_cont'].isnull(),['jd_sub_statis_del_rate']] =train_data[train_data['jd_sub_type'].str.contains('小众')][['jd_sub_statis_del_rate']].values[0]
        
       
        
        cur_jd_action_series = train_data.groupby(['cur_jd_type'])['browsed'].count()
        cur_jd_action =pd.DataFrame({'cur_jd_type':cur_jd_action_series.index, 'cur_jd_pos_cont':cur_jd_action_series.values})
        cur_jd_action['cur_jd_statis_del_rate'] = train_data.groupby(['cur_jd_type'])['statis_del'].mean().values
        train_data = pd.merge(train_data,cur_jd_action,how="left",on="cur_jd_type")
        valid_data = pd.merge(valid_data,cur_jd_action,how="left",on="cur_jd_type")
        valid_data.loc[valid_data['cur_jd_pos_cont'].isnull(),['cur_jd_statis_del_rate']]=train_data[train_data['cur_jd_type'].str.contains('其他')][['cur_jd_statis_del_rate']].values[0]
        
        
        '''
        
        desire_jd_industry_action_series = train_data.groupby(['desire_jd_type_id'])['browsed'].count()
        desire_jd_industry_action =pd.DataFrame({'desire_jd_type_id':desire_jd_industry_action_series.index, 'desire_jd_industry_pos_cont':desire_jd_industry_action_series.values})
        desire_jd_industry_action['desire_jd_industry_statis_del_rate'] = train_data.groupby(['desire_jd_type_id'])['statis_del'].mean().values
        train_data = pd.merge(train_data,desire_jd_industry_action,how="left",on="desire_jd_type_id")
        valid_data = pd.merge(valid_data,desire_jd_industry_action,how="left",on="desire_jd_type_id")
        valid_data.loc[valid_data['desire_jd_industry_pos_cont'].isnull(),['desire_jd_industry_statis_del_rate']]=desire_jd_industry_action[desire_jd_industry_action['desire_jd_type_id'].str.contains('小众')][['desire_jd_industry_statis_del_rate']].values[0]


     
     
        desire_jd_type_action_series = train_data.groupby(['desire_jd_type_id'])['browsed'].count()
        desire_jd_type_action =pd.DataFrame({'desire_jd_type_id':desire_jd_type_action_series.index, 'desire_jd_type_pos_cont':desire_jd_type_action_series.values})
        desire_jd_type_action['desire_jd_type_statis_del_rate'] = train_data.groupby(['desire_jd_type_id'])['statis_del'].mean().values
        train_data = pd.merge(train_data,desire_jd_type_action,how="left",on="desire_jd_type_id")
        valid_data = pd.merge(valid_data,desire_jd_type_action,how="left",on="desire_jd_type_id")
        valid_data.loc[valid_data['desire_jd_type_pos_cont'].isnull(),['desire_jd_type_statis_del_rate']] =desire_jd_type_action[desire_jd_type_action['desire_jd_type_id'].str.contains('小众')][['desire_jd_type_statis_del_rate']].values[0]
       
        '''

        

        jd_title_series = train_data.groupby(['jd_title'])['browsed'].count()
        jd_title_action =pd.DataFrame({'jd_title':jd_title_series.index, 'cur_jd_pos_cont':jd_title_series.values})
        jd_title_action['jd_title_statis_del_rate'] = train_data.groupby(['jd_title'])['statis_del'].mean().values
        train_data = pd.merge(train_data,jd_title_action,how="left",on="jd_title")
        valid_data = pd.merge(valid_data,jd_title_action,how="left",on="jd_title")
        valid_data.loc[valid_data['jd_title'].isnull(),['jd_title_statis_del_rate']]=jd_title_action[jd_title_action['jd_title'].str.contains('小众')][['jd_title_statis_del_rate']].values[0]




        
        
        result = feature_select(target,train_data,valid_data,test_big_table)
        t_x,t_y = result[0]
        v_x,v_y = result[1]
        test_x,_ = result[2]
        model = lgb.LGBMRanker(**param)
        print("Fold",idx,"-"*30)
        model.fit(t_x,t_y,group=train_group,eval_set=[(t_x,t_y),(v_x,v_y)],eval_group=[train_group,valid_group],early_stopping_rounds=100,verbose=10,
                  callbacks=[lgb.reset_parameter(learning_rate=lambda x: 0.03)]
                 )
        models.append(model)
        test_pred += model.predict(test_x)/n_folds
        scores.append(model.best_score_["valid_1"]["map@1"])
    print("mean score",np.mean(scores))
    return models,test_pred

In [254]:
train_big_table.shape

(506175, 68)

In [255]:
models,pred = cross_validate(target="mix_sat",param=dict(n_estimators=1000,metric="map",subsample=0.6,min_split_gain=10,colsample_bytree=0.6,max_depth=7,importance_type="gain",device="gpu",gpu_platform_id=0,gpu_device_id=0),n_folds=5)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Fold 0 ------------------------------
Training until validation scores don't improve for 100 rounds.
[10]	training's map@1: 0.263348	valid_1's map@1: 0.193548
[20]	training's map@1: 0.286986	valid_1's map@1: 0.201335
[30]	training's map@1: 0.289766	valid_1's map@1: 0.215795
[40]	training's map@1: 0.296719	valid_1's map@1: 0.211346
[50]	training's map@1: 0.300056	valid_1's map@1: 0.21802
[60]	training's map@1: 0.303949	valid_1's map@1: 0.221357
[70]	training's map@1: 0.314238	valid_1's map@1: 0.213571
[80]	training's map@1: 0.3198	valid_1's map@1: 0.209121
[90]	training's map@1: 0.322303	valid_1's map@1: 0.213571
[100]	training's map@1: 0.32842	valid_1's map@1: 0.21802
[110]	training's map@1: 0.333704	valid_1's map@1: 0.215795
[120]	training's map@1: 0.335929	valid_1's map@1: 0.214683
[130]	training's map@1: 0.338988	valid_1's map@1: 0.215795
[140]	training's map@1: 0.346774	valid_1's map@1: 0.219132
[150]	training's map@1: 0.348721	valid_1's map@1: 0.219132
Early stopping, best iterati

In [256]:
import matplotlib.pyplot as plt
import seaborn as sns

feature_importance = pd.DataFrame()
for idx,model in enumerate(models):
    fold_importance = pd.DataFrame()
    fold_importance["feature"] = ["live_city_id","desire_jd_salary_id","cur_industry_id","cur_jd_type","cur_salary_id",
             "cur_degree_id","birthday","city","jd_sub_type","require_nums","jd_sub_statis_del_rate","cur_jd_statis_del_rate","desire_jd_industry_statis_del_rate",
                    "start_month","jd_title_statis_del_rate",
             "min_salary","is_travel","min_years","min_edu_level",
             "desire_jd_city_1","desire_jd_city_2","desire_jd_city_3","exp_in_desc_num",
                   "desire_min_salary","desire_max_salary","salary_large_than_desire","cur_salary_min",
                   "cur_salary_max","salary_large_than_cur","job_description_len","experience_num","work_years_statisfied","work_years","desire_jd_city_num","desire_salary_diff"]+cross_feature_names

    fold_importance["importance"] = model.feature_importances_
    fold_importance["fold"] = idx
    feature_importance = pd.concat([feature_importance,fold_importance])
plt.figure(figsize=(16, 12));
sns.barplot(x="importance", y="feature", data=feature_importance.sort_values(by="importance", ascending=False));
plt.title('mix_satsatisfied LGB Features (avg over folds)');

ValueError: Length of values does not match length of index

In [94]:
train_big_table_tmp = pd.merge(train_big_table,jd_sub_action,how="left",on="jd_sub_type")
train_big_table_tmp = pd.merge(train_big_table_tmp,cur_jd_action,how="left",on="cur_jd_type")
#train_big_table_tmp = pd.merge(train_big_table_tmp,desire_jd_type_action,how="left",on="desire_jd_type_id")
train_big_table_tmp = pd.merge(train_big_table_tmp,jd_title_action,how="left",on="jd_title")

In [95]:
from feature_selector import FeatureSelector
feature_list =["live_city_id","desire_jd_salary_id","cur_industry_id","cur_jd_type","cur_salary_id",
             "cur_degree_id","birthday","city","jd_sub_type","require_nums","jd_sub_statis_del_rate","cur_jd_statis_del_rate",
                    "start_month","jd_title_statis_del_rate",
             "min_salary","is_travel","min_years","min_edu_level",
             "desire_jd_city_1","desire_jd_city_2","desire_jd_city_3","exp_in_desc_num",
                   "desire_min_salary","desire_max_salary","salary_large_than_desire","cur_salary_min",
                   "cur_salary_max","salary_large_than_cur","job_description_len","experience_num","work_years_statisfied","work_years","desire_jd_city_num","desire_salary_diff"]+cross_feature_names



train_fr =train_big_table_tmp[feature_list]
fs = FeatureSelector(data = train_fr, labels = train_big_table['mix_sat'])
fs.identify_collinear(correlation_threshold=0.8)
fs.record_collinear.head()

5 features with a correlation magnitude greater than 0.80.



,drop_feature,corr_feature,corr_value
0,desire_max_salary,desire_min_salary,0.860420
1,cur_salary_max,cur_salary_min,0.868241
2,work_years,birthday,0.925685
3,desire_jd_city_num,desire_jd_city_1,0.955741
4,desire_salary_diff,desire_max_salary,0.938315


In [257]:
test_action = pd.read_csv("data/zhaopin_round1_user_exposure_B_20190819",delim_whitespace=True)
test_action.head()

,user_id,jd_no
0,167827d429af352a93b28666c33ed0fd,6855780caa3e8b4c6c65adfacdda5282
1,167827d429af352a93b28666c33ed0fd,20603e8eec8f4fbab757ad70e86fdec2
2,167827d429af352a93b28666c33ed0fd,b612c4ec69756fe4944071cb0525ef8d
3,167827d429af352a93b28666c33ed0fd,0558a2371bbfa13eb9295f8011eeb997
4,167827d429af352a93b28666c33ed0fd,fde5a7db457d1123f56be0e295576c0c


In [258]:
submit = test_big_table[["user_id","jd_no"]]
submit["score"] = pred
submit = submit.reset_index(drop=True)
result = pd.merge(test_action,submit,how="left",on=["user_id","jd_no"])
result.fillna(-100,inplace=True)
result = result.groupby("user_id",as_index=False).apply(lambda x:x.sort_values("score",ascending=False))
result[["user_id","jd_no","score"]].to_csv("submission2_x.csv",index=False)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [259]:
submit = test_big_table[["user_id","jd_no"]]
submit["score"] = pred
submit = submit.reset_index(drop=True)
result = pd.merge(test_action,submit,how="left",on=["user_id","jd_no"])
result.fillna(-100,inplace=True)
result = result.groupby("user_id",as_index=False).apply(lambda x:x.sort_values("score",ascending=False))
# 需要对结果进行去重
result[["user_id","jd_no","score"]].drop_duplicates().to_csv("submission2_x.csv",index=False)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [260]:
result[["user_id","jd_no","score"]].drop_duplicates().shape

(29674, 3)

In [ ]:
#work_years_statisfied=1
#is_edu_ok？ 1
#year&is_city_eq 